In [20]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json

In [21]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'a3d44f08074e235aa594a7da869e77aa', 'Date': 'Thu, 22 Aug 2024 03:05:05 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data= json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and well-structured. She explains complex concepts clearly."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant professor! Makes programming fun and accessible. Always available for extra help.'},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Knowledgeable but sometimes rushes through material. Office hours are helpful.'},
 {'professor': 'Prof. David Brown',
  'subject': 'History',
  'stars': 4,
  'review': 'Passionate about the subject. Assignments are challenging but fair.'},
 {'professor': 'Dr. Lisa Martinez',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Excellent at breaking down complex theories. Lab sessions are well-organized.'},
 {'professor': 'Prof. James Wilson',
  'subject': 'Economics',
  'stars': 2,
  'review': 'Lectures can be dry. Grading is

In [22]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [23]:
processed_data[0]

{'values': [0.0038522093,
  0.0036571608,
  0.03151985,
  0.021091253,
  0.009661406,
  0.003096396,
  0.0017651896,
  0.053989444,
  0.0051427805,
  0.01871166,
  0.042468574,
  -0.009414344,
  -0.01859463,
  0.01786645,
  0.01903674,
  0.027254786,
  -0.050634608,
  -0.020285051,
  0.05409347,
  0.06870911,
  0.044757146,
  -0.018776676,
  0.03245608,
  -0.0008159532,
  -0.031337805,
  -0.041220263,
  0.015200785,
  0.011091762,
  0.025759414,
  0.004213049,
  0.06163535,
  0.0025453838,
  -0.023054741,
  -0.042832665,
  -0.025382321,
  0.027280793,
  -0.0029663637,
  -0.0011686661,
  0.02092221,
  0.018321563,
  0.024719156,
  0.0025697649,
  -0.048996203,
  -0.01667015,
  0.05255909,
  0.021039238,
  0.00975893,
  -0.010201041,
  0.04972438,
  0.044991203,
  -0.033756405,
  0.0066836644,
  0.005123276,
  -0.024966218,
  -0.03128579,
  -0.017047245,
  0.006235053,
  0.046733636,
  -0.015577879,
  -0.018503608,
  0.044028964,
  -0.0048697125,
  -0.01598098,
  0.011481859,
  -0.027540

In [24]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}